In [4]:
#load data 
import pandas as pd
import numpy as np
import requests 
import tarfile
import os 
import gensim
import gensim.downloader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

url = "https://www.cs.jhu.edu/~mdredze/datasets/sentiment/processed_acl.tar.gz"
filename = "processed_acl"
extension = ".tar.gz"

response = requests.get(url, allow_redirects=True)

print("http response:",response.status_code,response.reason)

with open(filename+extension,"wb") as file: 
    file.write(response.content)
    
tar = tarfile.open(filename+extension, "r:gz")
tar.extractall()
tar.close()
# # merge files using shell commands 
print(os.popen("cat " + filename+"/books/negative.review > mixed.txt").read())
print(os.popen("cat " + filename+"/books/positive.review >> mixed.txt").read())
# print(os.popen("cat " + filename+"/dvd/all.review >> mixed.txt").read())
# print(os.popen("cat " + filename+"/dvd/positive.review >> mixed.txt").read())



# parse data into array for word 2 vect 
dataArray = []
labelArray = []

with open("mixed.txt", "r") as datafile: 
    lines = datafile.readlines()
    for e in lines:
        lineList= []
        splitLine = e.split()
        label = splitLine.pop(-1)
        for i in splitLine:
            dictSplit = i.split(':')
            lineList.append(dictSplit[0])
        dataArray.insert(-1, lineList)
        labelArray.insert(-1, (label.split(':')[1]))

#download word2vec model          
word2vec = gensim.downloader.load('fasttext-wiki-news-subwords-300')


# create mean embedding 
embed = np.array([ np.mean([word2vec[w] for w in words if w in word2vec], axis=0) for words in dataArray])

X_train, X_test, y_train, y_test = train_test_split(embed, labelArray, test_size=0.2, random_state=42)

text_classifier = RandomForestClassifier(n_estimators=700, random_state=42)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))



http response: 200 OK


[[164  35]
 [ 37 164]]
              precision    recall  f1-score   support

    negative       0.82      0.82      0.82       199
    positive       0.82      0.82      0.82       201

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

0.82


In [ ]:
#  dl tubspam data set 
# load into numpy array 
# create embedings on full text 
# use classifier to classify the dataset 
